In [ ]:
from sportsreference.mlb.teams import Roster

giants = Roster('SFG', year = '2010', slim=True)
giants.players

In [4]:
from sportsreference.mlb.roster import Player
buster_posey = Player('poseybu01')
buster_posey.dataframe[["at_bats", "hits", "batting_average", "doubles", "triples", "home_runs", "runs_batted_in"], index='career']

,at_bats,hits,batting_average,doubles,triples,home_runs,runs_batted_in
2009,17,2,0.118,0,0,0,0
2010,406,124,0.305,23,2,18,67
2011,162,46,0.284,5,0,4,21
2012,530,178,0.336,39,1,24,103
2013,520,153,0.294,34,1,15,72
2014,547,170,0.311,28,2,22,89
2015,557,177,0.318,28,0,19,95
2016,539,155,0.288,33,2,14,80
2017,494,158,0.320,34,0,12,67
2018,398,113,0.284,22,1,5,41


In [1]:
# pull in dependencies needed from sportsreference api
from sportsreference.mlb.teams import Teams
from sportsreference.mlb.teams import Roster

# to create a list of player ids:
# set up an empty array that we will append to
player_list = []

# 1) create a loop to go through years
for year in range(2019,2020):

# 2) create a loop to go through teams and pull out the player names and their ids
    for team in Teams(year):
        player_data = Roster(team.abbreviation, year = year, slim=True).players
        player_list.append(player_data)

In [9]:
# convert the dictionay of arrays to an array of arrays using list comprehension
info = [list(x.keys()) for x in player_list]

In [10]:
# flatten the list using itertools
import itertools

merged = list(itertools.chain(*info))

In [11]:
merged

['chiriro01',
 'gourryu01',
 'altuvjo01',
 'correca01',
 'bregmal01',
 'brantmi02',
 'marisja01',
 'reddijo01',
 'alvaryo01',
 'springe01',
 'whitety01',
 'diazal02',
 'kempto01',
 'strawmy01',
 'stassma01',
 'maldoma01',
 'toroab01',
 'tuckeky01',
 'mayfija01',
 'fishede01',
 'stubbga01',
 'colege01',
 'mileywa01',
 'urquijo01',
 'verlaju01',
 'greinza01',
 'biagijo01',
 'guduare01',
 'sanchaa01',
 'mchugco01',
 'perezci01',
 'rodgebr01',
 'pressry01',
 'martico02',
 'jamesjo02',
 'devench02',
 'harriwi10',
 'smithjo05',
 'armenro01',
 'peacobr01',
 'valdefr01',
 'osunaro01',
 'abreubr01',
 'sneedcy01',
 'rondohe01',
 'barneau01',
 'muncyma01',
 'hernaen02',
 'seageco01',
 'turneju01',
 'pederjo01',
 'polloaj01',
 'bellico01',
 'tayloch03',
 'verdual01',
 'beatyma01',
 'martiru01',
 'smithwi05',
 'freesda01',
 'luxga01',
 'negrokr01',
 'riosed01',
 'garliky01',
 'gyorkje01',
 'whitety01',
 'galero01',
 'darnatr01',
 'ryuhy01',
 'buehlwa01',
 'kershcl01',
 'maedake01',
 'stripro01',
 '